**<h1>Tanish Chhabra<br>
101916038<br>
CS10</h1>**

In [25]:
import numpy as np
import pandas as pd
import cudf
import cupy

**<h2>Q.1</h2>**

**<h3>Using numpy, pandas and sklearn</h3>**

In [26]:
#Importing Libraries 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [27]:
%%time

#Importing Dataset
df = pd.read_csv('../datasets/USA_Housing (1).csv')

#Dividing dataset into X(input variables) and Y(output variables)
X = df.iloc[:,:5]
Y = df.iloc[:,5]

#Scaling Input variables
ss = StandardScaler()
X_scaled = ss.fit_transform(X)
x = np.insert(X_scaled,0,1,axis=1)

#Dividing input and output features into 5 folds
beta = []
r2 = -1

no_of_folds = 5
n = len(X)
one_fold = n//no_of_folds
index = np.arange(0,n,1)

for i in range(no_of_folds):
    print("Iteration: ",i+1)
    
    test_index = np.arange(one_fold*i,one_fold*(i+1),1)
    train_index = np.delete(index,test_index)
    
    x_train, x_test, y_train, y_test = x[train_index], x[test_index], Y[train_index], Y[test_index]

    a = x_train.T.dot(x_train)
    b = np.linalg.inv(a)
    c = b.dot(x_train.T)
    beta_temp = c.dot(y_train)
    print("Beta: ",beta_temp)

    #Predicting Y
    y_predicted = x_test.dot(beta_temp)

    #Calculating error, sse, mse, rmse
    error = y_predicted - y_test
    sse = np.sum(error**2)
    mse = sse/len(y_test)
    rmse = np.sqrt(mse)
    print("SSE:",sse,",MSE:",mse,",RMSE:",rmse)

    #Calculating R2
    y_mean = np.mean(y_test)
    total_variance = np.sum((y_test-y_mean)**2)
    r2_temp = 1-sse/total_variance  
    print("R2 score:",r2_temp)

    #Optimised R2
    if r2_temp >= r2:
        r2 = r2_temp
        beta = beta_temp
    print("\n")


print("Max R2 score:",r2)
print("Corresponding Beta matrix:",beta,"\n")

#Testing 30% data with optimal value of beta
x_train, x_test, y_train, y_test = train_test_split(x,Y,test_size=0.30,random_state=42)

#Predicting Y
y_predicted = x_test.dot(beta)
y_predicted

#Calculating error,sse,mse,rmse and R2 score
error = y_predicted - y_test
sse = np.sum(error**2)
mse = sse/len(y_test)
rmse = np.sqrt(mse)
print("SSE:",sse,"\nMSE:",mse,"\nRMSE:",rmse)
print()

y_mean = np.mean(y_test)
total_variance = np.sum((y_test-y_mean)**2)
r2_temp = 1-sse/total_variance  
print("R2 score:",r2_temp,"\n")

Iteration:  1
Beta:  [1231872.07704612  229923.70318635  164052.98579011  120923.57426579
    3533.77470973  151855.58542488]
SSE: 10087104444118.805 ,MSE: 10087104444.118805 ,RMSE: 100434.57793070475
R2 score: 0.9175899480765108


Iteration:  2
Beta:  [1232586.40453402  228365.32287054  165201.14881927  122539.46855211
    1697.03433458  150055.81710562]
SSE: 10728109153419.246 ,MSE: 10728109153.419247 ,RMSE: 103576.58593243574
R2 score: 0.9203015496401128


Iteration:  3
Beta:  [1231448.09063371  230730.20810582  163554.55706976  121669.08713233
    2620.25215054  150923.12489298]
SSE: 9861616305461.945 ,MSE: 9861616305.461945 ,RMSE: 99305.67106395256
R2 score: 0.9152429915320014


Iteration:  4
Beta:  [1.23144707e+06 2.29921558e+05 1.64523054e+05 1.19737507e+05
 1.12425659e+03 1.51317802e+05]
SSE: 10176314743926.252 ,MSE: 10176314743.926252 ,RMSE: 100877.72174234633
R2 score: 0.9208503836977653


Iteration:  5
Beta:  [1233028.38536436  230778.59921985  163755.13607252  121885.135122

**<h3>Using CuDF, CuPy and CuML</h3>**

In [28]:
from cuml.preprocessing import StandardScaler
from cuml.model_selection import train_test_split

In [29]:
%%time

#Importing Dataset
df = cudf.read_csv('../datasets/USA_Housing (1).csv')
df.info()

#Dividing dataset into X(input variables) and Y(output variables)
X = df.iloc[:,:5]
Y = df.iloc[:,5]
Y = cupy.asarray(Y)

#Scaling Input variables
ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_scaled.insert(0,-1,1.0)
X_scaled = cupy.fromDlpack(X_scaled.to_dlpack())
x = X_scaled

#Dividing input and output features into 5 folds
beta = []
r2 = -1

no_of_folds = 5
n = len(X)
one_fold = n//no_of_folds
index = np.arange(0,n,1)

for i in range(no_of_folds):
    print("Iteration: ",i+1)
    
    test_index = np.arange(one_fold*i,one_fold*(i+1),1)
    train_index = np.delete(index,test_index)
    
    x_train, x_test, y_train, y_test = x[train_index], x[test_index], Y[train_index], Y[test_index]

    a = x_train.T.dot(x_train)
    b = cupy.linalg.inv(a)
    c = b.dot(x_train.T)
    beta_temp = c.dot(y_train)
    print("Beta: ",beta_temp)

    #Predicting Y
    y_predicted = x_test.dot(beta_temp)

    #Calculating error, sse, mse, rmse
    error = y_predicted - y_test
    sse = cupy.sum(error**2)
    mse = sse/len(y_test)
    rmse = cupy.sqrt(mse)
    print("SSE:",sse,",MSE:",mse,",RMSE:",rmse)

    #Calculating R2
    y_mean = cupy.mean(y_test)
    total_variance = cupy.sum((y_test-y_mean)**2)
    r2_temp = 1-sse/total_variance  
    print("R2 score:",r2_temp)

    #Optimised R2
    if r2_temp >= r2:
        r2 = r2_temp
        beta = beta_temp
    print("\n")


print("Max R2 score:",r2)
print("Corresponding Beta matrix:",beta,"\n")

#Testing 30% data with optimal value of beta
x_train, x_test, y_train, y_test = train_test_split(x,Y,test_size=0.30,random_state=42)

#Predicting Y
y_predicted = x_test.dot(beta)
y_predicted

#Calculating error,sse,mse,rmse and R2 score
error = y_predicted - y_test
sse = np.sum(error**2)
mse = sse/len(y_test)
rmse = np.sqrt(mse)
print("SSE:",sse,"\nMSE:",mse,"\nRMSE:",rmse)

y_mean = np.mean(y_test)
total_variance = np.sum((y_test-y_mean)**2)
r2_temp = 1-sse/total_variance  
print("R2 score:",r2_temp,"\n")

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   Avg. Area Income              5000 non-null   float64
 1   Avg. Area House Age           5000 non-null   float64
 2   Avg. Area Number of Rooms     5000 non-null   float64
 3   Avg. Area Number of Bedrooms  5000 non-null   float64
 4   Area Population               5000 non-null   float64
 5   Price                         5000 non-null   float64
dtypes: float64(6)
memory usage: 234.4 KB
Iteration:  1
Beta:  [1231872.07704612  229923.70318635  164052.98579011  120923.57426579
    3533.77470973  151855.58542488]
SSE: 10087104444118.816 ,MSE: 10087104444.118816 ,RMSE: 100434.5779307048
R2 score: 0.9175899480765107


Iteration:  2
Beta:  [1232586.40453402  228365.32287054  165201.14881927  122539.46855211
    1697.03433458  150055.81710562]
SSE: 10728109153419.22 ,M

<h3>Total time using GPU accelerated code - 40.8ms<br>
Total time without using GPU accelerated code - 24.2ms</h3>

**<h2>Q.2</h2>**

**<h3>Using sklearn</h3>**

In [30]:
#Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [31]:
%%time
df = pd.read_csv('../datasets/USA_Housing (1).csv')

X = df.iloc[:,:5]
Y = df.iloc[:,5]


ss = StandardScaler()
X_scaled = ss.fit_transform(X)
x = np.insert(X_scaled,0,1,axis=1)


x_train, x_test, y_train, y_test = train_test_split(x,Y,test_size=0.30,random_state=42)

model = LinearRegression()
model.fit(x_train,y_train)

predicted = model.predict(x_test)

print(metrics.r2_score(y_test, predicted),"\n")

0.9146818498916267 

CPU times: user 11.9 ms, sys: 0 ns, total: 11.9 ms
Wall time: 11.4 ms


**<h3>Using cuml</h3>**

In [32]:
from cuml.preprocessing import StandardScaler
from cuml.model_selection import train_test_split
from cuml.linear_model import LinearRegression
from cuml import metrics

In [33]:
%%time
df = cudf.read_csv('../datasets/USA_Housing (1).csv')

X = df.iloc[:,:5]
Y = df.iloc[:,5]

ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_scaled.insert(0,-1,1.0)
#X_scaled = cupy.fromDlpack(X_scaled.to_dlpack())
x = X_scaled

x_train, x_test, y_train, y_test = train_test_split(x,Y,test_size=0.30,random_state=42)

model = LinearRegression()
model.fit(x_train,y_train)

predicted = model.predict(x_test)

print(metrics.r2_score(y_test, predicted),"\n")

0.9143798851361878 

CPU times: user 33 ms, sys: 3.61 ms, total: 36.6 ms
Wall time: 35.8 ms


<h3>Total time using GPU accelerated code - 36.6ms<br>
Total time without using GPU accelerated code - 11.9ms</h3>